#### Libraries importeren

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.nn import functional as F

#### Dataset inladen en inzien

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [3]:
image, label = train_data[0]
print(f"Afbeelding formaat: {image.shape}")

Afbeelding formaat: torch.Size([1, 28, 28])


#### Architectuur van CNN bouwen

In [5]:
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Flatten(),
    
    nn.Linear(128 * 3 * 3, 128),
    nn.ReLU(),
    
    nn.Linear(128, 10)
)

#### Check voor GPU
I.v.m. dat het model zwaar kan zijn, check ik eerst ff of het huidige apparaat een GPU heeft. Dit kan het trainen een stuk versneller namelijk.

In [6]:
component = torch.device("cuda" if torch.cuda.is_available()
                         else "cpu")
model = model.to(component)
print(f"Model runt op {component}")

Model runt op cpu


#### Model trainen

In [37]:
optimalisator = optim.Adam(model.parameters(), lr=0.001)
criterium = nn.CrossEntropyLoss()

In [34]:
epochs = 10
for epoch in range(epochs):
    model.train()
    totale_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(component), labels.to(component) #Runt het model op geschikte component
        
        optimalisator.zero_grad()
        output = model(images)
        loss = criterium(output, labels)
        loss.backward()
        optimalisator.step()
        totale_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {totale_loss / len(train_loader)}")

torch.save(model.state_dict(), "Getrainde_modellen/CNN_Model_gewichten.pth")

Epoch 1, Loss: 0.16537782840623355
Epoch 2, Loss: 0.042823933943947734
Epoch 3, Loss: 0.029818433837822157
Epoch 4, Loss: 0.02302939241672536
Epoch 5, Loss: 0.017971442791060603
Epoch 6, Loss: 0.014190953837274808
Epoch 7, Loss: 0.01313107954007643
Epoch 8, Loss: 0.011758114205347617
Epoch 9, Loss: 0.008418667611160295
Epoch 10, Loss: 0.009530031548584646


#### Laad getrainde model indien nodig (voor apparaten zonder cuda bijvoorbeeld)

In [ ]:
model.load_state_dict(torch.load("Getrainde_modellen/CNN_Model_gewichten.pth", 
                                 map_location=torch.device(component)))


<All keys matched successfully>

#### Evaluatie van model